# 1차, 2차 학습 시도 실패했다.
# 모델 자체가 한계 있는 건 아닐까? 
# 모델을 바꿔보자. 

In [81]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(20)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_46 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_97 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_14  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_12 (Dropout)        (None, 2048)              0         
                                                                 
 dense_13 (Dense)            (None, 20)                409

In [82]:
# 이미지 데이터 전처리 # 데이터 증식 사용

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/train', 
    target_size=(150, 150),
    batch_size=100, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/flickr/test',
    target_size=(150,150),
    batch_size=50,
    class_mode='categorical'
)

Found 7993 images belonging to 20 classes.
Found 1999 images belonging to 20 classes.


In [83]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='val_categorical_accuracy', # 모델 검증 손실 모니터링
        patience= 20), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=10
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [84]:
import keras.optimizers

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = [keras.metrics.CategoricalAccuracy()]
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 80, 
    epochs=15,
    validation_data = validation_generator, 
    validation_steps=40, 
    verbose=2,
    callbacks=callbacks_list 
)

Epoch 1/15


2022-02-17 12:26:10.365633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 12:28:43.899119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 - 171s - loss: 2.5442 - categorical_accuracy: 0.2317 - val_loss: 2.2189 - val_categorical_accuracy: 0.3397 - lr: 0.0010 - 171s/epoch - 2s/step
Epoch 2/15
80/80 - 174s - loss: 2.1429 - categorical_accuracy: 0.3451 - val_loss: 2.1385 - val_categorical_accuracy: 0.3557 - lr: 0.0010 - 174s/epoch - 2s/step
Epoch 3/15
80/80 - 169s - loss: 2.0243 - categorical_accuracy: 0.3771 - val_loss: 2.1109 - val_categorical_accuracy: 0.3552 - lr: 0.0010 - 169s/epoch - 2s/step
Epoch 4/15
80/80 - 172s - loss: 1.9434 - categorical_accuracy: 0.3982 - val_loss: 2.0895 - val_categorical_accuracy: 0.3627 - lr: 0.0010 - 172s/epoch - 2s/step
Epoch 5/15
80/80 - 158s - loss: 1.8921 - categorical_accuracy: 0.4111 - val_loss: 2.0716 - val_categorical_accuracy: 0.3767 - lr: 0.0010 - 158s/epoch - 2s/step
Epoch 6/15
80/80 - 161s - loss: 1.8436 - categorical_accuracy: 0.4269 - val_loss: 2.0791 - val_categorical_accuracy: 0.3732 - lr: 0.0010 - 161s/epoch - 2s/step
Epoch 7/15
80/80 - 162s - loss: 1.8159 - categorica

# 미세조정

In [85]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(1e-5),  # Low learning rate
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)


history = model.fit_generator(
    train_generator,
    steps_per_epoch = 80, 
    epochs=200,
    validation_data = validation_generator, 
    validation_steps=40, 
    verbose=2,
    callbacks=callbacks_list 
)

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_46 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_97 (Bat  (None, 150, 150, 3)      12        
 chNormalization)                                                
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_14  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_12 (Dropout)        (None, 2048)              0         
                                                                 
 dense_13 (Dense)            (None, 20)                409

2022-02-17 13:07:33.502734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-17 13:12:58.425582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 - 344s - loss: 1.5614 - categorical_accuracy: 0.5147 - val_loss: 2.0397 - val_categorical_accuracy: 0.3912 - lr: 1.0000e-05 - 344s/epoch - 4s/step
Epoch 2/200
80/80 - 408s - loss: 1.4674 - categorical_accuracy: 0.5316 - val_loss: 2.0342 - val_categorical_accuracy: 0.4007 - lr: 1.0000e-05 - 408s/epoch - 5s/step
Epoch 3/200
80/80 - 399s - loss: 1.4188 - categorical_accuracy: 0.5485 - val_loss: 2.0229 - val_categorical_accuracy: 0.3997 - lr: 1.0000e-05 - 399s/epoch - 5s/step
Epoch 4/200
80/80 - 403s - loss: 1.3628 - categorical_accuracy: 0.5662 - val_loss: 2.0290 - val_categorical_accuracy: 0.4022 - lr: 1.0000e-05 - 403s/epoch - 5s/step
Epoch 5/200
80/80 - 327s - loss: 1.3190 - categorical_accuracy: 0.5826 - val_loss: 2.0430 - val_categorical_accuracy: 0.4037 - lr: 1.0000e-05 - 327s/epoch - 4s/step
Epoch 6/200
80/80 - 327s - loss: 1.2699 - categorical_accuracy: 0.5945 - val_loss: 2.0494 - val_categorical_accuracy: 0.4072 - lr: 1.0000e-05 - 327s/epoch - 4s/step
Epoch 7/200
80/80 - 32

KeyboardInterrupt: 

# Xception 모델도 40% 이상 검증 성능 향상되지 않았다. 